# ⚠️ WE NEED TO PRODIGY 🦄 TO FIX THE ANNOTATION
For example, `201210379231` is not a CHEMICAL

In [10]:
# download prodigy for your system: https://gerdes.fr/saclay/informationRetrieval/prodigy/
# put the files in a subfolder
# try what works. for me it's:
# !pip install ./mac/prodigy-1.11.11-cp310-cp310-macosx_11_0_arm64.whl

## Prepare the patent text in prodigy format

In [18]:
import json

with open('C07K.txt', 'r') as f:
    texts = f.read().strip()
# split text in smaller chunks, then put in chunk in a list of dicts for prodigy
texts = texts.split('\n')
texts = [text.strip() for text in texts]

size = 1500
new_texts = []
for text in texts:
    #if text is too long, split it
    if len(text) > size:
        #split text in chunks
        chunks = [text[i:i+size] for i in range(0, len(text), size)]
        #add chunks to list
        new_texts.extend(chunks)
    elif len(text) > 0:
        new_texts.append(text)

with open('C07K.jsonl', 'w') as outfile:
    for i in range(len(new_texts)):
        if i < 300000:
            continue
        json.dump({'text': new_texts[i]}, outfile)
        outfile.write('\n')

## Fix annotations with prodigy + ktgiahieu/bert-for-patents-finetuned-ner

In [23]:
!prodigy bert.ner.manual ner_combine ./C07K.jsonl --hide-wp-prefix -F ner_bert_patent_manualv2.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Added dataset ner_C07K_last2 to database SQLite.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

⚠ Front End Log - 2023-04-23 10:55:01+00:00: Duplicate _task_hash found
in Frontend batch.
^C

✔ Saved 266 annotations to database SQLite
Dataset: ner_C07K_last2
Session ID: 2023-04-23_12-42-59



## Output annotations in Prodigy format (for SpaCy)

In [55]:
!prodigy db-out ner_combine ner_combine.jsonl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
✔ Created dataset 'ner_combine' in database SQLite
✔ Imported 777 annotations to 'ner_combine' (session
2023-04-23_13-30-30) in database SQLite
Found and keeping existing "answer" in 777 examples


## Output annotations in Huggingface format

In [59]:
import json
import spacy

from prodigy.components.db import connect

db = connect()
prodigy_annotations = db.get_dataset("ner_combine_filtered")
examples = ((eg["text"], eg) for eg in prodigy_annotations)
nlp = spacy.blank("en")

dataset = []

for doc, eg in nlp.pipe(examples, as_tuples=True):
    if eg['answer']=='ignore':
        continue
    try:
        doc.ents = [doc.char_span(s["start"], s["end"], s["label"]) for s in eg["spans"]]
        iob_tags = [f"{t.ent_iob_}-{t.ent_type_}" if t.ent_iob_ else "O" for t in doc]
        iob_tags = [t.strip("-") for t in iob_tags]
        tokens = [str(t) for t in doc]
        temp_data = {
            "tokens": tokens,
            "tags": iob_tags
        }
        dataset.append(temp_data)
    except:
        pass

with open('data.jsonl', 'w') as outfile:
    for entry in dataset:
        if any([x!='O' for x in entry['tags']]):
            json.dump(entry, outfile)
            outfile.write('\n')



# Finally, let's fine-tune the model again using these new annotations:
- Upload `data.jsonl` to Huggingface Hub
- (You can use our annotated data directly `https://huggingface.co/datasets/ktgiahieu/IR_combined_filtered`)
- Fine-tune another model with the finetune notebook 📓: https://colab.research.google.com/drive/1OzCY782KJSF0FBDS0d1CoMhfp3-RtJMV